### Building an agent executor with human in the loop
In this example, we will be building a chat exector that has a human in the loop. We will use the human to approve specific actions. This is similar to the work we did during my research project

In [1]:
import os
from dotenv import load_dotenv
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.messages import BaseMessage, AIMessage, FunctionMessage, HumanMessage
from langgraph.prebuilt import ToolExecutor, ToolInvocation 
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph, END

In [2]:
load_dotenv()

True

In [3]:
tools = [TavilySearchResults(max_results=2)]
tool_executor = ToolExecutor(tools)
functions = [ format_tool_to_openai_function(tool) for tool in tools]

In [4]:
model = ChatOpenAI(model="mistralai/Mixtral-8x7B-Instruct-v0.1")
model = model.bind_functions(functions)

In [5]:
from typing import Sequence, Annotated, TypedDict
import operator

In [6]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    approved: Annotated[str, operator.setitem] = False

In [7]:
import json

In [12]:
def call_model(state: AgentState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

def call_tool(state: AgentState):
    messages = state['messages']
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    fn = last_message.additional_kwargs["tool_calls"][0]["function"]
    action = ToolInvocation(
        tool=fn["name"],
        tool_input=json.loads(fn["arguments"]),
    )
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

def call_human(state: AgentState):
    messages = state['messages']
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    
    fn = last_message.additional_kwargs["tool_calls"][0]["function"]
    print("The agent would like to call tool : ", fn["name"])
    print("With arguments : ", fn["arguments"])
    
    response = input("would you like to approve this operation y or n")
    
    if response == "y":
        return {"approved": True}
    
    elif response == "n":
        return {"approved": False}
    
    else:
        raise ValueError("Invalid approval message sent by human")
    
    
def handle_approval(state: AgentState):
    if state["approved"]:
        return "continue"
    else:
        return "reject"
    
def reject_action(state: AgentState):
    messages = state['messages']
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    
    fn = last_message.additional_kwargs["tool_calls"][0]["function"]
    
    human_message = HumanMessage(content=f"I would not like you to call tool {fn['name']} with arguments {fn['arguments']} generate a response with the information you currently have")
    
    return {"messages": [human_message]}
    

def should_continue(state: AgentState):
    messages = state['messages']
    last_message = messages[-1]
    # If there is no function call, then we finish
    if "tool_calls" not in last_message.additional_kwargs:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"

In [13]:
workflow = StateGraph(AgentState)


workflow.add_node("agent", call_model)
workflow.add_node("human", call_human)
workflow.add_node("reject", reject_action)
workflow.add_node("tool", call_tool)


workflow.add_edge("tool", "agent")
workflow.add_edge("reject", "agent")

workflow.add_conditional_edges("agent", should_continue, {
    "end": END,
    "continue": "human"
})

workflow.add_conditional_edges("human", handle_approval, {
    "continue": "tool",
    "reject": "reject"
})

workflow.set_entry_point("agent")

app = workflow.compile()

In [15]:
from pprint import pprint

In [16]:
inputs = {"messages": [HumanMessage(content="What is the weather in birmingham, uk?",)]}


for output in app.stream(inputs):
    for key, value in output.items():
        print(f"from node {key}")
        print("----")
        pprint(f"{value}")
    print("----")

from node agent
----
("{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': "
 "'call_smq0dx0iouoke5d1qejym2kp', 'function': {'arguments': "
 '\'{"query":"weather in Birmingham, UK"}\', \'name\': '
 "'tavily_search_results_json'}, 'type': 'function'}]})]}")
----
The agent would like to call tool :  tavily_search_results_json
With arguments :  {"query":"weather in Birmingham, UK"}
from node human
----
"{'approved': True}"
----
from node tool
----
("{'messages': [FunctionMessage(content='[{\\'url\\': "
 '\\\'https://www.weatherapi.com/\\\', \\\'content\\\': "Weather in '
 "Birmingham, UK is {\\'location\\': {\\'name\\': \\'Birmingham\\', "
 "\\'region\\': \\'West Midlands\\', \\'country\\': \\'United Kingdom\\', "
 "\\'lat\\': 52.49, \\'lon\\': -1.86, \\'tz_id\\': \\'Europe/London\\', "
 "\\'localtime_epoch\\': 1708043019, \\'localtime\\': \\'2024-02-16 0:23\\'}, "
 "\\'current\\': {\\'last_updated_epoch\\': 1708042500, \\'last_updated\\': "
 "\\'2024-02-16 00:15\